**This is example of how to trace model with jit and export it to the onnx**

In [ ]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

In [ ]:
ray.init(object_store_memory=1024*1024*1000)

In [ ]:
config_name = '../rl_games/configs/ppo_pendulum_torch.yaml'

In [ ]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['params']['config']['normalize_input'] = True
    config['params']['config']['max_epochs'] = 200
    config['params']['config']['full_experiment_name'] = 'pendulum_onnx'
    config['params']['config']['save_best_after'] = 10
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

In [ ]:
class ModelWrapper(torch.nn.Module):
    '''
    Main idea is to ignore outputs which we don't need from model
    '''
    def __init__(self, model):
        torch.nn.Module.__init__(self)
        self._model = model
        
        
    def forward(self,input_dict):
        input_dict['obs'] = self._model.norm_obs(input_dict['obs'])
        '''
        just model export doesn't work. Looks like onnx issue with torch distributions
        thats why we are exporting only neural network
        '''
        #print(input_dict)
        #output_dict = self._model.a2c_network(input_dict)
        #input_dict['is_train'] = False
        #return output_dict['logits'], output_dict['values']
        return self._model.a2c_network(input_dict)

In [ ]:
agent = runner.create_player()
agent.restore('runs/pendulum_onnx/nn/pendulum.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states,
}

with torch.no_grad():
    adapter = flatten.TracingAdapter(ModelWrapper(agent.model), inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "pendulum.onnx", verbose=True, input_names=['obs'], output_names=['mu','log_std', 'value'])

onnx_model = onnx.load("pendulum.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
ort_model = ort.InferenceSession("pendulum.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 3)).astype(np.float32)},
)
print(outputs)

In [ ]:
is_done = False
env = agent.env
obs = env.reset()
prev_screen = env.render(mode='rgb_array')
plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    mu = outputs[0].squeeze(1)
    sigma = np.exp(outputs[1].squeeze(1))
    action = np.random.normal(mu, sigma)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    plt.imshow(screen)
    display.display(plt.gcf())    
    display.clear_output(wait=True)
print(total_reward, num_steps)
ipythondisplay.clear_output(wait=True)